<a href="https://colab.research.google.com/github/prachi735/EVA5/blob/main/Assignment5/Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [9]:
dropout_value = 0.5
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3) #input-28 Output-26  RF 1
        self.bn1 = nn.BatchNorm2d(16)
        self.do1 = nn.Dropout(0.5)

        self.conv2 = nn.Conv2d(16, 16, 3) #input-26 Output-24 RF
        self.bn2 = nn.BatchNorm2d(16)
        self.do2 = nn.Dropout(0.1)

        self.pool1 = nn.MaxPool2d(2, 2) #input-22 Output-11 RF

        self.conv4 = nn.Conv2d(16, 32, 3) #input-11 Output-9 RF
        self.bn4 = nn.BatchNorm2d(32)
        self.do4 = nn.Dropout(0.1)

        self.conv5 = nn.Conv2d(32, 32, 3) #input-9 Output-7 RF
        self.bn5 = nn.BatchNorm2d(32)
        self.do5 = nn.Dropout(0.1)
        
        self.conv6 = nn.Conv2d(32, 16, 1) #input-7 Output-5 RF
        self.bn6 = nn.BatchNorm2d(16)
        self.do6 = nn.Dropout(0.1)

        self.conv7 = nn.Conv2d(16, 16, 3) #input-5 Output-3 RF
        self.bn7 = nn.BatchNorm2d(16)
        self.do7 = nn.Dropout(0.1)

        self.conv8 = nn.Conv2d(16, 10, 3) #input-3 Output-1 RF
        self.gvp = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.do1(x)

        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.do2(x)

        x = self.pool1(x)

        x = F.relu(self.conv4(x))
        x = self.bn4(x)
        x = self.do4(x)

        x = F.relu(self.conv5(x))
        x = self.bn5(x)
        x = self.do5(x)

        x = F.relu(self.conv6(x))
        x = self.bn6(x)
        x = self.do6(x)

        x = F.relu(self.conv7(x))
        x = self.bn7(x)
        x = self.do7(x)

        x = F.relu(self.conv8(x))
        x = self.gvp(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [10]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]           4,640
       BatchNorm2d-9           [-1, 32, 10, 10]              64
          Dropout-10           [-1, 32, 10, 10]               0
           Conv2d-11             [-1, 32, 8, 8]           9,248
      BatchNorm2d-12             [-1, 32, 8, 8]              64
          Dropout-13             [-1, 32, 8, 8]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [12]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0940, Accuracy: 9711/10000 (97.1%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0516, Accuracy: 9832/10000 (98.3%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0211, Accuracy: 9939/10000 (99.4%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0217, Accuracy: 9936/10000 (99.4%)



epoch=14 loss=0.006815207656472921 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 31.26it/s]



Test set: Average loss: 0.0217, Accuracy: 9932/10000 (99.3%)

